For each SELECTED result, create snippet...

# Snippet should include:
1. document title
2. 2 sentences from doc with highest COSINE SIMILARITY w/respect to q. 


In [1]:
from classes.inverted_index import inverted_index
from classes.persist_index_memory import persist_index_memory

In [2]:
storage = persist_index_memory()
i_i = inverted_index(storage=storage)

persist_index_memory.py INFO: 2019-10-07 16:52:02,987: Instantiating storage object
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
inverted_index.py INFO: 2019-10-07 16:52:03,255: Instantiating inverted index object


# 1. load dataset

In [3]:
'''
load original dataset, the doc collection 
'''

import pandas as pd

prefix = "~/Documents/Classes/CS537_IR/project1/chunks/"
sample_path = prefix + 'chunk_17.pkl' 
doc_collection = pd.read_pickle(sample_path)
DC = doc_collection.to_dict('records')
# use pandas read_csv(skiprows=FUNCTION)

# 2. create index from dataset

In [4]:
#load inverted index
# path = "../../../Documents/Classes/CS537_IR/project1/"

import pickle
from classes.inverted_index import inverted_index, Appearance

with open('inverted_index_chunk17.pkl', 'rb') as handle:
    i_i = pickle.load(handle)

In [5]:
# i_i.create_index(DC, process_text=True)

# import pickle
# with open('inverted_index_chunk17.pkl', 'wb') as handle:
#     pickle.dump(i_i, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 3. given query, find relevant results 

In [6]:
def return_matches(CR={}, len_DC=1699999):
    doc_first = {}
    
    # better way of generating pertinent indices?
    for i in range(1, len_DC+1):
        doc_first[i] = []    

    for token in CR:
        for val in CR[token]:
            match = ('match: ' + str(token), val.frequency)
            doc_first[val.docId].append(match)
    return {k: doc_first[k] for k in doc_first if len(doc_first[k]) > 0}

def match_scaling(CR = {}, match_num = int):
    return {k: CR[k] for k in CR if len(CR[k]) == match_num}

In [7]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import logging

nltk.download('stopwords')
stemer = PorterStemmer()
tokenizer = TweetTokenizer()

def stem(tokens=''):
    return [stemer.stem(w) for w in tokens]
def tokenize(text=''):
    return tokenizer.tokenize(text)
def remove_stopwords(tokens=[]):
    stop_words = set(stopwords.words('english'))
    return [w for w in tokens if not w in stop_words]
def remove_capitalization(tokens=[]):
    return [w.lower() for w in tokens]
def remove_punctuation(text=''):
    return re.sub(r'[^\w\s]', '', text)
def preprocess(text=''):
    # Remove punctuation from the text.
    text = remove_punctuation(text=text)
    # Tokenize
    tokens = tokenize(text=text)
    # Remove stop words
    tokens = remove_stopwords(tokens=tokens)
    # Remove capitalization
    tokens = remove_capitalization(tokens=tokens)
    # Stem terms
    tokens = stem(tokens=tokens)
    return tokens

[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
query = "what time is shark week"
q = preprocess(query)
q = ' '.join(q)

CR = i_i.lookup_query(q)

In [9]:
CR = return_matches(CR)

In [10]:
# select resources that match every token from query
CR = match_scaling(CR, len(q.split()))

In [11]:
CR

{1602897: [('match: time', 3), ('match: shark', 1), ('match: week', 15)],
 1618493: [('match: time', 1), ('match: shark', 1), ('match: week', 1)],
 1619433: [('match: time', 2), ('match: shark', 1), ('match: week', 1)],
 1628933: [('match: time', 3), ('match: shark', 1), ('match: week', 1)],
 1638449: [('match: time', 4), ('match: shark', 1), ('match: week', 3)],
 1648033: [('match: time', 2), ('match: shark', 1), ('match: week', 2)],
 1649133: [('match: time', 1), ('match: shark', 1), ('match: week', 2)],
 1651158: [('match: time', 3), ('match: shark', 1), ('match: week', 1)],
 1652709: [('match: time', 5), ('match: shark', 1), ('match: week', 1)],
 1661061: [('match: time', 3), ('match: shark', 7), ('match: week', 1)],
 1662393: [('match: time', 11), ('match: shark', 3), ('match: week', 2)]}

# 4. rank relevant results

In [12]:
from classes.ranking import ranking

r_ranking = ranking()
resources = list(CR.keys())

In [13]:
# max_freq=[]
# for docId, matches in CR.items():
#     tmp_freq = 0
#     for token in matches:

#         frequency = token[1]
#         if frequency > tmp_freq:
# #             print(token[0])
#             tmp_freq = frequency
# #             print(tmp_freq)
#     max_freq.append(tmp_freq)
# max_freq

In [19]:
CR.items()

dict_items([(1602897, [('match: time', 3), ('match: shark', 1), ('match: week', 15)]), (1618493, [('match: time', 1), ('match: shark', 1), ('match: week', 1)]), (1619433, [('match: time', 2), ('match: shark', 1), ('match: week', 1)]), (1628933, [('match: time', 3), ('match: shark', 1), ('match: week', 1)]), (1638449, [('match: time', 4), ('match: shark', 1), ('match: week', 3)]), (1648033, [('match: time', 2), ('match: shark', 1), ('match: week', 2)]), (1649133, [('match: time', 1), ('match: shark', 1), ('match: week', 2)]), (1651158, [('match: time', 3), ('match: shark', 1), ('match: week', 1)]), (1652709, [('match: time', 5), ('match: shark', 1), ('match: week', 1)]), (1661061, [('match: time', 3), ('match: shark', 7), ('match: week', 1)]), (1662393, [('match: time', 11), ('match: shark', 3), ('match: week', 2)])])

In [22]:
processed_query_tokens = q.split()
processed_query_tokens

['time', 'shark', 'week']

In [ ]:
# finding the most frequent use of a term in the collection
terms = list(self.index.keys())
stemmed_tokens=[]
for id, document in relevant_results.items():
    text = self.preprocessing.remove_punctuation(text=document['content'])
    tokens = self.preprocessing.tokenize(text=text)
    tokens = self.preprocessing.remove_stopwords(tokens=tokens)
    tokens = self.preprocessing.remove_capitalization(tokens=tokens)
    stems = self.preprocessing.stem(tokens=tokens)
    for stem in stems:
        found = False
        for token in tokens:
            if token == stem:
                found = True
        if found == False:
            stemmed_tokens.append(stem)
# We have now all the stemmed words that didn't match documents, now
# let's make a unique list of them.
stem_tokens = list(set(stemmed_tokens))

# Now let's generate the matrix
self.doc_term_matrix_all=[]
for id, document in self.storage.index.items():
    term_doc_matrix=dict()
    text = self.preprocessing.remove_punctuation(text=document['text'])
    tokens = self.preprocessing.tokenize(text=text)
    tokens = self.preprocessing.remove_stopwords(tokens=tokens)
    tokens = self.preprocessing.remove_capitalization(tokens=tokens)
    stems = self.preprocessing.stem(tokens=tokens)
    for full_stem in stem_tokens:
        found = False
        for doc_stemmed_token in stems:
            if full_stem == doc_stemmed_token:
                found = True
        if found == True:
            term_doc_matrix[full_stem] = 1
        else:
            term_doc_matrix[full_stem] = 0
    self.doc_term_matrix_all.append(term_doc_matrix)

In [ ]:
# We have now all the stemmed words that didn't match documents, now
# let's make a unique list of them.
'''
^???
'''

In [18]:
# this step is necessary for idf
i_i.create_term_document_matrix()

KeyboardInterrupt: 

In [15]:
max_freq = r_ranking.get_max_frequencies(index=CR) 

100%|██████████| 11/11 [00:00<00:00, 73350.31it/s]


In [16]:
i_i.storage.max_frequency_terms_per_doc = max_freq

In [17]:
res = r_ranking.relevance_ranking(query = 'state nation disput local employ the',
                           num_results=5,
                            index=i_i.index,
                            resources=resources,
                            max_freq=i_i.storage.max_frequency_terms_per_doc,
                            N=len(i_i.storage.index),
                            term_doc_matrix=i_i.doc_term_matrix_all)

AttributeError: 'inverted_index' object has no attribute 'doc_term_matrix_all'

# calculate cosine similarity between every 'sentence' in doc and q

## retrieve original, unprocessed doc-string

In [ ]:
original = dc[1]['content']

In [ ]:
original

## get doc title

In [ ]:
'''
cool python trick:
https://stackoverflow.com/questions/31426095/assign-multiple-values-of-a-list
'''

title, *text = original.split('\n\n')
text = ' '.join(text)

In [ ]:
title

In [ ]:
import nltk.data

sent_split = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = sent_split.tokenize(text)
sentences[0]

## pre-process sentences same way as index.

In [ ]:
sent_tokens = [preprocess(sentences[i]) for i in range(len(sentences))]

# Calculate cosine similarity between sentences & the query

In [ ]:
term_doc_weights = []
term_query_weights = []

# let the query be...
query = "post office hours"

## retrieve weights for terms in sentence, + in query

In [ ]:
sent = sent_tokens[0] 
for w in sent:
    print(w)

In [ ]:
index = i_i
query = "what time is shark week"

-----------------

----------------

In [ ]:
# params: self, query='', num_results=5, index=[], resources=[]
'''
Calculate relevance ranking
query: String containing the query, this is a single string and it is not tokenized
resources: Array of document id's that will be processed
return: Array of ranked documents.
'''

ranker.relevance_ranking()

In [ ]:
res = ranker.relevance_ranking(query = q,
                           num_results=5,
                            index=i_i.index,
                            resources=resources,
                            max_freq=i_i.storage.max_frequency_terms_per_doc,
                            N=len(i_i.storage.index),
                            term_doc_matrix=i_i.doc_term_matrix_all)